# Modelo 4 - Heurística Final

### Leer los datos de excel

In [2]:
import pandas as pd

#Funcion para leer los datos de los pacientes
def leer_datos():
    # Leer el archivo de excel
    df = pd.read_excel('DT4.xlsx', sheet_name='Pacientes') #!!!! Se cambia el nombre de la hoja
    df1 = pd.read_excel("DT4.xlsx", sheet_name="Medicos") #!!!! Se cambia el nombre de la hoja
    df3 = pd.read_excel("DT4.xlsx", sheet_name="Clientes")
    
    # Crear un diccionario para almacenar los datos de los pacientes y otro para los medicos
    pacientesDict = {}
    medicos = {}
    Clientes = {}
    
    # diccionario con los tipos de cliente y el tiempo de atencion esperado
    for index, row in df3.iterrows():
        llave = row["Tipo"]
        valor = row["Tiempo"]
        Clientes[llave] = valor


    # Iterar sobre las filas del DataFrame
    for index, row in df.iterrows():
        llave = row["ID Servicio"]
        valor1 = row["Llegada"]
        valor2 = row["Atencion"]
        valor3 = row["Latitud"]
        valor4 = row["Longitud"]
        valor5 = row["Entidad"]

        # Agregar valores a la lista correspondiente en el diccionario
        if llave in pacientesDict:
            pacientesDict[llave].append([valor1, valor2, valor3, valor4, valor5])
        else:
            pacientesDict[llave] = [int(valor1), int(valor2), valor3, valor4, str(valor5)]
    pacientes = pacientesDict

    # Iterar sobre las filas del DataFrame
    for index, row in df1.iterrows():
        llave = row["ID Medico"]
        Disponibiidad = 0 
        UltOcupado = 0
        NumSol = 0
        lat = 4.681230925773865
        lon = -74.06200456196707
        Inicioj = row["Inicio Jornada"]
        Finj = row["Fin Jornada"]
    
        # Agregar valores a la lista correspondiente en el diccionario
        if llave in medicos:
            medicos[llave].append([Disponibiidad, UltOcupado, lat, lon, NumSol, Inicioj, Finj])
        else:
            medicos[llave] = [int(Disponibiidad), int(UltOcupado),lat, lon, NumSol, int(Inicioj), int(Finj)]
    doctores = medicos
    
    return doctores, pacientes, Clientes

doctores = leer_datos()[0]
pacientes = leer_datos()[1]
Clientes = leer_datos()[2]

### Calcular la distancia/tiempo entre cada par de nodos

In [3]:
from math import radians, cos, sin, asin, sqrt
def distancia_tierra(lat1, lat2, lon1, lon2):
    
    # Convertir grados a radianes.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Formula Haversine, para calcular la distancia entre dos puntos de una esfera dadas sus coordenadas de longitud y latitud
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    
    # Radio de la tierra en km.
    r = 6371
    
    respuesta = ((c * r)/24.9)*60
    # Calcular la distancia final en Km.
    return respuesta

### Cambiar datos para Gantt

In [4]:
import datetime

def convertMin(minutos):
    # Obtenemos la fecha de hoy
    fecha_actual = datetime.date.today()
    # Combinamos la fecha de hoy con la hora 0:00:00
    fecha_hora = datetime.datetime.combine(fecha_actual, datetime.time.min) + datetime.timedelta(minutes=minutos)
    # Convertimos la fecha y hora a formato de cadena
    fecha_hora_str = fecha_hora.strftime("%Y-%m-%d %H:%M:%S")
    return fecha_hora_str

### Regla de Prioridad

In [5]:
# Regla de prioridad
def prioridad(NumSolicitudes, Tdesplazamiento, Mdescoupa):
    # Si el número de solicitudes esta entre 0 y 2
    if NumSolicitudes >= 0 and NumSolicitudes <= 2:
        beta = 0.55
        alpha = 0.45
    # Si el número de solicitudes esta entre 3 y 4
    elif NumSolicitudes >= 3 and NumSolicitudes <= 4:
        beta = 0.65
        alpha = 0.35
    # Si el número de solicitudes esta entre 5 o más
    elif NumSolicitudes >= 5:
        beta = 0.75
        alpha = 0.25

    # Calculamos la prioridad
    pesoPrioridad = NumSolicitudes * alpha + Tdesplazamiento + Mdescoupa * beta

    return pesoPrioridad

### Asignación de los médicos

In [6]:
global doctores

import pandas as pd

# Función para asignar doctor a paciente
def asignar_doctor(atencion,latitud, longitud, minuto, gantt, IDservicio, TipoCliente):

    # Lista de posible asignacion
    posibleAsignacion = []

    # Actualizar estado de los médicos
    if minuto > 0:
        for i in doctores:
            # Si el minuto actual es mayor al ultimo ocupado significa que el médico esta disponible
            if minuto > doctores[i][1]:
                # Actualizar estado del médico
                doctores[i][0] = 0
    
    for i in doctores:
        # Si el médico esta en su jornada laboral y esta disponible
        if minuto >= doctores[i][5] and minuto <= doctores[i][6] and doctores[i][0] == 0:
            # Agregar médico y su distancia a la lista de posible asignacion

            # Encontrar distancia de los médicos al paciente
            distancia = distancia_tierra(doctores[i][2], latitud, doctores[i][3], longitud)
            # Agregar médico y su distancia a la lista de posible asignacion
            posibleAsignacion.append([i, distancia])
    
    # Si no hay ningún médico disponible
    if len(posibleAsignacion) == 0:
        # Se buscan médicos que esten en su jornada laboral para asignarles el paciente usando la regla de prioridad

        for i in doctores:
            # Si el doctor esta en su jornada laboral
            if minuto >= doctores[i][5] and minuto <= doctores[i][6]:
                # Se aplican las reglas de prioridad

                # Calcular tiempo de desplazamiento
                Tdesplazamiento = distancia_tierra(doctores[i][2], latitud, doctores[i][3], longitud)
                # Calcular tiempo de desocupacion
                Mdescoupa = doctores[i][1]
                # Calcular prioridad
                pesoPrioridad = prioridad(doctores[i][4], Tdesplazamiento, Mdescoupa)
                # Agregar a la lista de posible asignacion
                posibleAsignacion.append([i, pesoPrioridad])
        

        # Si existen médicos en su jornada laboral
        if len(posibleAsignacion) > 0:
            # Escoger el médico con mayor prioridad, es decir, el que tenga menor peso

            # Ordenar la lista de posible asignacion
            posibleAsignacion.sort(key=lambda x: x[1])
            # Asignar el doctor con menor prioridad
            doc_Asignar = posibleAsignacion[0][0]

            # Actualizar variables del médico asignado
            Mdescoupa = doctores[doc_Asignar][1]
            Tdesplazamiento = distancia_tierra(doctores[doc_Asignar][2], latitud, doctores[doc_Asignar][3], longitud)

            # Actualizar el gantt
            gantt.append({'Solicitud': IDservicio, 'Médico': doc_Asignar, 'Inicio': convertMin(Mdescoupa + Tdesplazamiento), 'Fin': convertMin(Mdescoupa + Tdesplazamiento + atencion), 'Distancia': Tdesplazamiento, 'Inicio Jornada': doctores[doc_Asignar][5], 'Fin Jornada': doctores[doc_Asignar][6], 'Inicio traslado': Mdescoupa, 'Fin traslado': Mdescoupa + Tdesplazamiento, 'Llegada solicitud': minuto})
            print("Inicio",convertMin(Mdescoupa + Tdesplazamiento),"Fin", convertMin(Mdescoupa + Tdesplazamiento + atencion))
            
            # Actualizar el estado del doctor
            doctores[doc_Asignar][0] = 1
            doctores[doc_Asignar][1] = Mdescoupa + Tdesplazamiento + atencion
            doctores[doc_Asignar][2] = latitud
            doctores[doc_Asignar][3] = longitud
            doctores[doc_Asignar][4] += 1
            print("Asignación tipo 1: El médico asignado es: ", doc_Asignar, " Se asigno porque no esta disponible y era el de menor prioridad en el minuto: ", minuto, "Al servicio: ", IDservicio)
        else:
            # No hay médicos disponibles
            print("No hay médicos disponibles")
                

    # Si un solo médico esta disponible
    elif len(posibleAsignacion) == 1:
        # Se asignará solo si cumple con el tiempo de servicio y no tiene más de 5 servicios asignados

        # Calcular tiempo de desplazamiento
        Tdesplazamiento = distancia_tierra(doctores[posibleAsignacion[0][0]][2], latitud, doctores[posibleAsignacion[0][0]][3], longitud)
        # Si el médico cumple con el tiempo de servicio y no tiene más de 5 servicios asignados
        if minuto + Tdesplazamiento <= minuto + Clientes[TipoCliente] and doctores[posibleAsignacion[0][0]][4] < 5:
            # Se asigna el médico

            # Asignar el doctor
            doc_Asignar = posibleAsignacion[0][0]
            # Actualizar variables del médico asignado
            Mdescoupa = doctores[doc_Asignar][1]
            Tdesplazamiento = distancia_tierra(doctores[doc_Asignar][2], latitud, doctores[doc_Asignar][3], longitud)
            
            # Actualizar el gantt
            gantt.append({'Solicitud': IDservicio, 'Médico': doc_Asignar, 'Inicio': convertMin(minuto + Tdesplazamiento), 'Fin': convertMin(minuto + Tdesplazamiento + atencion), 'Distancia': Tdesplazamiento, 'Inicio Jornada': doctores[doc_Asignar][5], 'Fin Jornada': doctores[doc_Asignar][6], 'Inicio traslado': minuto, 'Fin traslado': minuto + Tdesplazamiento, 'Llegada solicitud': minuto})
            print("Inicio",convertMin(minuto + Tdesplazamiento),"Fin", convertMin(minuto + Tdesplazamiento + atencion))

            # Actualizar el estado del doctor
            doctores[doc_Asignar][0] = 1
            doctores[doc_Asignar][1] = minuto + Tdesplazamiento + atencion
            doctores[doc_Asignar][2] = latitud
            doctores[doc_Asignar][3] = longitud
            doctores[doc_Asignar][4] += 1
            print("Asignación tipo 2: El médico asignado es: ", doc_Asignar, " Se asigno porque era el único disponible, cumplia con el tiempo de servicio y no había atendido 5 solicitudes en el minuto: ", minuto, "Al servicio: ", IDservicio)
        
        # Si el médico no cumple con el tiempo de servicio o tiene más de 5 servicios asignados
        else:
            # Se buscan médicos que esten en su jornada laboral para asignarles el paciente usando la regla de prioridad (Pueden estar ocupados o no)

            # Vacia la lista de posible asignacion
            posibleAsignacion = []

            # Buscar médicos que esten en su jornada laboral
            for i in doctores:
                # Si el doctor esta en su jornada laboral
                if minuto >= doctores[i][5] and minuto <= doctores[i][6]:
                    # Se aplican las reglas de prioridad

                    # Calcular tiempo de desplazamiento
                    Tdesplazamiento = distancia_tierra(doctores[i][2], latitud, doctores[i][3], longitud)
                    # Calcular tiempo de desocupacion
                    Mdescoupa = doctores[i][1]
                    # Calcular prioridad
                    pesoPrioridad = prioridad(doctores[i][4], Tdesplazamiento, Mdescoupa)
                    # Agregar a la lista de posible asignacion
                    posibleAsignacion.append([i, pesoPrioridad])

            # Si existen médicos en su jornada laboral
            if len(posibleAsignacion) > 0:
                # Escoger el médico con mayor prioridad, es decir, el que tenga menor peso

                # Ordenar la lista de posible asignacion
                posibleAsignacion.sort(key=lambda x: x[1])
                # Asignar el doctor con menor prioridad
                doc_Asignar = posibleAsignacion[0][0]
                # Actualizar variables del médico asignado
                Mdescoupa = doctores[doc_Asignar][1]
                Tdesplazamiento = distancia_tierra(doctores[doc_Asignar][2], latitud, doctores[doc_Asignar][3], longitud)

                # Si el médico esta ocupado
                if Mdescoupa > minuto:
                    # Actualizar el gantt
                    
                    gantt.append({'Solicitud': IDservicio, 'Médico': doc_Asignar, 'Inicio': convertMin(Mdescoupa + Tdesplazamiento), 'Fin': convertMin(Mdescoupa + Tdesplazamiento + atencion), 'Distancia': Tdesplazamiento, 'Inicio Jornada': doctores[doc_Asignar][5], 'Fin Jornada': doctores[doc_Asignar][6], 'Inicio traslado': Mdescoupa, 'Fin traslado': Mdescoupa + Tdesplazamiento, 'Llegada solicitud': minuto})
                    print("Inicio",convertMin(Mdescoupa + Tdesplazamiento),"Fin", convertMin(Mdescoupa + Tdesplazamiento + atencion))

                    # Actualizar el estado del doctor: Tiempo de desocupacion
                    doctores[doc_Asignar][1] = Mdescoupa + Tdesplazamiento + atencion
                # Si el médico esta desocupado
                else:
                    # Actualizar el gantt

                    gantt.append({'Solicitud': IDservicio, 'Médico': doc_Asignar, 'Inicio': convertMin(minuto + Tdesplazamiento), 'Fin': convertMin(minuto + Tdesplazamiento + atencion), 'Distancia': Tdesplazamiento, 'Inicio Jornada': doctores[doc_Asignar][5], 'Fin Jornada': doctores[doc_Asignar][6], 'Inicio traslado': Mdescoupa, 'Fin traslado': Mdescoupa + Tdesplazamiento, 'Llegada solicitud': minuto})
                    print("Inicio",convertMin(minuto + Tdesplazamiento),"Fin", convertMin(minuto + Tdesplazamiento + atencion))
                    # Actualizar el estado del doctor: Tiempo de desocupacion
                    doctores[doc_Asignar][1] = minuto + Tdesplazamiento + atencion

                # Actualizar otros estados del doctor
                doctores[doc_Asignar][0] = 1
                doctores[doc_Asignar][2] = latitud
                doctores[doc_Asignar][3] = longitud
                doctores[doc_Asignar][4] += 1
                print("Asignación tipo 3: El médico asignado es: ", doc_Asignar, " Se asignó porque el unico médico disponible no cumplia con el tiempo de servicio o ya habia atendido 5 solicitudes, en el minuto: ", minuto, "Al servicio: ", IDservicio)
            else:
                # No hay médicos disponibles
                print("No hay médicos disponibles")
    
    # Si hay más de un médico disponible
    else:
        # Se asignará el médico con menor prioridad de los disponibles
        for i in posibleAsignacion:
            # Si el doctor esta en su jornada laboral
            if minuto >= doctores[i[0]][5] and minuto <= doctores[i[0]][6]:
                # Calcular tiempo de desplazamiento
                Tdesplazamiento = i[1]

                # Si el médico cumple con el tiempo de servicio
                if minuto + Tdesplazamiento <= minuto + Clientes[TipoCliente]:

                    # Calcular prioridad
                    pesoPrioridad = prioridad(doctores[i[0]][4], Tdesplazamiento, doctores[i[0]][1])
                    # Cambiar la distancia por la prioridad
                    i[1] = pesoPrioridad
                else:
                    # Cambiar la distancia por un número muy grande
                    i[1] = 10000
        
        if len(posibleAsignacion) > 0:

            # Ordenar la lista de posible asignacion
            posibleAsignacion.sort(key=lambda x: x[1]) 
            # Asignar el doctor con menor prioridad
            doc_Asignar = posibleAsignacion[0][0]
            # Actualizar variables del médico asignado
            Mdescoupa = doctores[doc_Asignar][1]
            Tdesplazamiento = distancia_tierra(doctores[doc_Asignar][2], latitud, doctores[doc_Asignar][3], longitud)

            # Actualizar el gantt
            gantt.append({'Solicitud': IDservicio, 'Médico': doc_Asignar, 'Inicio': convertMin(minuto + Tdesplazamiento), 'Fin': convertMin(minuto + Tdesplazamiento + atencion), 'Distancia': Tdesplazamiento, 'Inicio Jornada': doctores[doc_Asignar][5], 'Fin Jornada': doctores[doc_Asignar][6], 'Inicio traslado': minuto, 'Fin traslado': minuto + Tdesplazamiento, 'Llegada solicitud': minuto})
            print("Inicio",convertMin(minuto + Tdesplazamiento),"Fin", convertMin(minuto + Tdesplazamiento + atencion))
            # Actualizar el estado del doctor
            doctores[doc_Asignar][0] = 1
            doctores[doc_Asignar][1] = minuto + Tdesplazamiento + atencion
            doctores[doc_Asignar][2] = latitud
            doctores[doc_Asignar][3] = longitud
            doctores[doc_Asignar][4] += 1
            print("Asignación tipo 4: El médico asignado es: ", doc_Asignar, " Se asignó porque estaba disponible, cumplia con la franja y tiempo de servicio el de menor prioridad en el minuto: ", minuto, "Al servicio: ", IDservicio)
        else:
            # No hay médicos disponibles
            print("No hay médicos disponibles")

    return gantt


### Simular Pacientes

In [7]:
import pandas as pd
import openpyxl

global doctores

# Función principal para asignar médicos a pacientes
def asignacion_doctor1():
    pacientes = leer_datos()[1]
    gantt = []

    # Ordenar dict de pacientes por minuto
    pacientes = dict(sorted(pacientes.items(), key=lambda item: item[1][0]))

    for i in pacientes:
        # Asignar doctor a paciente
        atencion = pacientes[i][1]
        minuto = pacientes[i][0]
        latitud = pacientes[i][2]
        longitud = pacientes[i][3]
        IDservicio = i
        TipoCliente = pacientes[i][4]
        gantt = asignar_doctor(atencion,latitud,longitud, minuto, gantt, IDservicio, TipoCliente)
    
    return gantt

In [8]:
gantt = asignacion_doctor1()

Inicio 2023-05-27 00:06:07 Fin 2023-05-27 00:20:07
Asignación tipo 4: El médico asignado es:  1032491705  Se asignó porque estaba disponible, cumplia con la franja y tiempo de servicio el de menor prioridad en el minuto:  1 Al servicio:  324231
Inicio 2023-05-27 01:04:09 Fin 2023-05-27 01:23:09
Asignación tipo 4: El médico asignado es:  1015405667  Se asignó porque estaba disponible, cumplia con la franja y tiempo de servicio el de menor prioridad en el minuto:  41 Al servicio:  324232
Inicio 2023-05-27 01:20:01 Fin 2023-05-27 01:56:01
Asignación tipo 2: El médico asignado es:  1032491705  Se asigno porque era el único disponible, cumplia con el tiempo de servicio y no había atendido 5 solicitudes en el minuto:  51 Al servicio:  324233
Inicio 2023-05-27 02:12:02 Fin 2023-05-27 02:42:02
Asignación tipo 4: El médico asignado es:  1015405667  Se asignó porque estaba disponible, cumplia con la franja y tiempo de servicio el de menor prioridad en el minuto:  124 Al servicio:  324236
Inicio 

In [9]:
# Contar cuantos pacientes atendio cada doctor exclueyndo los que inician su jornada a las 0
for i in doctores:
    if doctores[i][5] != 0:
        print("El doctor", i, "atendio", doctores[i][4], "pacientes")

# calcular el promedio de pacientes atendidos por cada doctor
promedio = 0
d = 0
for i in doctores:
    if doctores[i][5] != 0:
        promedio += doctores[i][4]
        d += 1

promedio = promedio / d
print("El promedio de pacientes atendidos por cada doctor es:", promedio)
# sumar el total de pacientes atendidos
total = 0
for i in doctores:
    total += doctores[i][4]

print("El total de pacientes atendidos es:", total)


El doctor 1018472151 atendio 10 pacientes
El doctor 1015437933 atendio 8 pacientes
El doctor 80073352 atendio 10 pacientes
El doctor 52997773 atendio 10 pacientes
El doctor 79955886 atendio 11 pacientes
El doctor 1015414697 atendio 7 pacientes
El doctor 1019088914 atendio 11 pacientes
El doctor 1020777651 atendio 7 pacientes
El doctor 1018446151 atendio 7 pacientes
El doctor 80185764 atendio 8 pacientes
El doctor 39779707 atendio 9 pacientes
El doctor 1127250183 atendio 1 pacientes
El doctor 52200795 atendio 9 pacientes
El doctor 1024468225 atendio 8 pacientes
El doctor 1121853934 atendio 7 pacientes
El doctor 1016039086 atendio 7 pacientes
El doctor 80383487 atendio 7 pacientes
El doctor 1020803066 atendio 8 pacientes
El doctor 57293715 atendio 8 pacientes
El doctor 1020808271 atendio 8 pacientes
El doctor 80727764 atendio 8 pacientes
El doctor 1098635342 atendio 8 pacientes
El doctor 1082996581 atendio 9 pacientes
El doctor 1117504115 atendio 9 pacientes
El doctor 1085310672 atendio 

### Gantt general

In [10]:
import plotly.express as px
import pandas as pd
#Poner los datos de la lista en un dataframe
df1 = pd.DataFrame(gantt)
# poner los datos en el formato que requiere plotly poner la columna solicitud como texto
df1['Solicitud'] = df1['Solicitud'].astype(str)
# poner la columna médico como texto
df1['Médico'] = df1['Médico'].astype(str)

fig = px.timeline(df1, x_start="Inicio", x_end="Fin", y="Médico", color="Médico",
                  hover_name="Solicitud",
                  title="Solicitudes atendidas por médicos")

fig.update_layout(xaxis_title="Tiempo", yaxis_title="Médico")
fig.update_yaxes(autorange="reversed")
fig.update_xaxes(rangeslider_visible=True)
# Hacer el grafico más grande para que se vea mejor
fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
)
fig.show()

In [11]:
import folium
from folium import plugins
from folium.plugins import HeatMap

# Crear un mapa centrado en la ciudad de Bogotá
mapa = folium.Map(location=[4.60971, -74.08175], zoom_start=12)

# Crear dict de coordenadas pacientes
coordenadas = {}

for i in pacientes:
    coordenadas[i] = [pacientes[i][2], pacientes[i][3]]

# Crear un mapa de calor con la ubicación de los pacientes
HeatMap(data=coordenadas.values(), radius=15).add_to(mapa)

# Crear un grupo de marcadores de pacientes
markers = folium.FeatureGroup(name='Pacientes')

# Agregar el marcador de cada paciente al grupo de marcadores
for i in pacientes:
    folium.Marker(
        location=[pacientes[i][2], pacientes[i][3]],
        popup="Paciente " + str(i),
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(markers)

# Crear un control de capas para mostrar u ocultar los marcadores de pacientes
folium.LayerControl().add_to(mapa)

# Mostrar el mapa
mapa

In [13]:
# Encontrar la cantidad de solicitudes atendidas por cada médico
def solicitudes_atendidas(gantt):
    solicitudes = {}

    for i in gantt:
        if i['Médico'] not in solicitudes:
            solicitudes[i['Médico']] = 1
        else:
            solicitudes[i['Médico']] += 1

    return solicitudes

solicitudes = solicitudes_atendidas(gantt)
print(solicitudes)

# Calcular la desviación estándar de las solicitudes atendidas por cada médico
def desviacion_estandar(solicitudes):
    # Calcular la media de las solicitudes atendidas por cada médico
    media = sum(solicitudes.values())/len(solicitudes)

    # Calcular la varianza de las solicitudes atendidas por cada médico
    varianza = sum([(i - media)**2 for i in solicitudes.values()])/len(solicitudes)

    # Calcular la desviación estándar de las solicitudes atendidas por cada médico
    desviacion = varianza**(1/2)

    return desviacion

desviacion = desviacion_estandar(solicitudes)
print(desviacion)

# calcular solicited promedio por médico
def solicitudes_promedio(solicitudes):
    promedio = sum(solicitudes.values())/len(solicitudes)
    return promedio

promedio = solicitudes_promedio(solicitudes)
print(promedio)

# Calcular total de solicitudes atendidas
def total_solicitudes(solicitudes):
    total = sum(solicitudes.values())
    return total

total = total_solicitudes(solicitudes)
print(total)

{1032491705: 5, 1015405667: 7, 1018472151: 10, 1015437933: 8, 80073352: 10, 52997773: 10, 79955886: 11, 1015414697: 7, 1019088914: 11, 1020777651: 7, 1018446151: 7, 80185764: 8, 39779707: 9, 1127250183: 1, 52200795: 9, 1024468225: 8, 1121853934: 7, 1016039086: 7, 80383487: 7, 1020803066: 8, 57293715: 8, 1020808271: 8, 80727764: 8, 1098635342: 8, 1082996581: 9, 1117504115: 9, 1085310672: 9, 1095825225: 8, 1014217039: 10, 1014266018: 9, 1098697055: 8, 85488148: 7, 80075437: 7, 1053327980: 6, 1083026203: 7, 1083012532: 6, 1018440480: 7, 1019074166: 7, 1140888504: 8}
1.7326200929770414
7.846153846153846
306
